In [19]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!pip install folium==0.5.0
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [20]:
CLIENT_ID = 'ZVJWN3ZVVBAVA1W35IUKR5DOY44K25Y3PEJSWGTTDGAZ45BO' # your Foursquare ID
CLIENT_SECRET = 'NS51X4MP40RSAN5VD04F42KUAZDEB444NFO5X4EI4UHLHD5Y' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZVJWN3ZVVBAVA1W35IUKR5DOY44K25Y3PEJSWGTTDGAZ45BO
CLIENT_SECRET:NS51X4MP40RSAN5VD04F42KUAZDEB444NFO5X4EI4UHLHD5Y


In [21]:
LIMIT = 500 # Maximum is 100
cities = ["New York, NY", 'Chicago, IL', 'San Francisco, CA', 'Boston, MA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d102951735") # AFRICAN CUISINE CATEGORY ID
    results[city] = requests.get(url).json()

In [22]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


The Foursquare API Only gives us the nearest 100 venues in the city.

Let's first check out their densities by our eyes

In [23]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of African cuisine in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of African cuisine in New York, NY =  329
Showing Top 100
Total number of African cuisine in Chicago, IL =  268
Showing Top 100
Total number of African cuisine in San Francisco, CA =  144
Showing Top 100
Total number of African cuisine in Boston, MA =  99
Showing Top 100


In [24]:
maps[cities[0]]

In [25]:
maps[cities[1]]


In [10]:
maps[cities[2]]

In [11]:
maps[cities[3]]

In [12]:
maps[cities[4]]

In [ ]:
We can see that New York and chicago are the most dense cities with African cuisine. And better than that, they are just one shore away.

However, Let's have a concrete measure of this density.

For this I will use some basic statistics. I will get the mean location of the pizza places which should be near to most of them if they are really dense or far if not.

Next I will take the average of the distance of the venues to the mean coordinates.

In [26]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

New York, NY
Mean Distance from Mean coordinates
0.02126691236977309
Chicago, IL
Mean Distance from Mean coordinates
0.06794817039179735
San Francisco, CA
Mean Distance from Mean coordinates
0.026899999395355305
Boston, MA
Mean Distance from Mean coordinates
0.017604801511467264


In [27]:
maps[cities[0]]

In [28]:
maps[cities[1]]

In [29]:
maps[cities[2]]

In [30]:
maps[cities[3]]

In [ ]:
We now see that New York is his best option. And as aplus the Third best place is san francisco which is just on the other side of the shore. Our tourist's best interest would be to book a hotel near that mean coordinate to surround himself with the 100 African cuisine there!!
